In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql import Column

In [3]:
spark = SparkSession.builder.master("local[2]").appName("AoC_2022_6[2]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/06 19:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/06 19:09:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [67]:
inputs = spark.read.schema("signals string").csv("./data/aoc_6.txt")

# inputs = spark.createDataFrame([("abcd", ), ("bvwbjplbgvbhsrlpgdmjqwftvncz", ), ("mjqjpqmgbljsphdztnvjfqwrcgsmlb", )], ("signals", ))

In [68]:
signals = F.col("signals")
marker_len = 14 # 14 for part B, 4 for part A
marker_length = F.lit(marker_len)


def distinct_check(val: Column) -> Column:
    splits = F.split(val, "")
    return F.size(splits) == F.size(F.array_distinct(splits))

def construct_accumulator(found: bool, position: int) -> Column:
    return F.struct(F.lit(found).alias("found"), F.lit(position).cast("int").alias("position"))

def compute_merge(acc: Column, x: Column) -> Column:
    current_window_signal = signals.substr(x, marker_length)
    is_distinct = distinct_check(current_window_signal)
    return F.when(is_distinct, construct_accumulator(True, (x + (marker_length - 1)))).otherwise(acc)
    # return current_window_signal

def merge(acc: Column, x: int) -> Column:
    return F.when(acc.found, acc).otherwise(compute_merge(acc, x)) # naive short circuit

acc_init = construct_accumulator(False, None)

signal_result = F.aggregate(F.sequence(F.lit(1), F.length("signals") - (marker_len- 1)), acc_init, merge)

In [69]:
inputs.select((signal_result.position).alias("result")).show(truncate=False)

+------+
|result|
+------+
|2334  |
+------+

